# [HoloViz](#HoloViz) stuff is at the bottom

# Preprocessing and clustering

In [ ]:
# Core scverse libraries
import scanpy as sc
import anndata as ad

# Data retrieval
import pooch

In [ ]:
sc.settings.set_figure_params(dpi=50, facecolor="white")

The data used in this basic preprocessing and clustering tutorial was collected from bone marrow mononuclear cells of healthy human donors and was part of [openproblem's NeurIPS 2021 benchmarking dataset](https://openproblems.bio/competitions/neurips_2021/) {cite}`Luecken2021`. The samples used in this tutorial were measured using the 10X Multiome Gene Expression and Chromatin Accessability kit. 


We are reading in the count matrix into an [AnnData](https://anndata.readthedocs.io/en/latest/tutorials/notebooks/getting-started.html) object, which holds many slots for annotations and different representations of the data.

In [ ]:
EXAMPLE_DATA = pooch.create(
    path=pooch.os_cache("scverse_tutorials"),
    base_url="doi:10.6084/m9.figshare.22716739.v1/",
)
EXAMPLE_DATA.load_registry_from_doi()

In [ ]:
%%time

samples = {
    "s1d1": "s1d1_filtered_feature_bc_matrix.h5",
    "s1d3": "s1d3_filtered_feature_bc_matrix.h5",
}
adatas = {}

for sample_id, filename in samples.items():
    path = EXAMPLE_DATA.fetch(filename)
    sample_adata = sc.read_10x_h5(path)
    sample_adata.var_names_make_unique()
    adatas[sample_id] = sample_adata

adata = ad.concat(adatas, label="sample")
adata.obs_names_make_unique()
print(adata.obs["sample"].value_counts())
adata

The data contains ~8,000 cells per sample and 36,601 measured genes. We'll now investigate these with a basic preprocessing and clustering workflow.

## Quality Control

The scanpy function {func}`~scanpy.pp.calculate_qc_metrics` calculates common quality control (QC) metrics, which are largely based on `calculateQCMetrics` from scater {cite}`McCarthy2017`. One can pass specific gene population to {func}`~scanpy.pp.calculate_qc_metrics` in order to calculate proportions of counts for these populations. Mitochondrial, ribosomal and hemoglobin genes are defined by distinct prefixes as listed below. 

In [ ]:
# mitochondrial genes, "MT-" for human, "Mt-" for mouse
adata.var["mt"] = adata.var_names.str.startswith("MT-")
# ribosomal genes
adata.var["ribo"] = adata.var_names.str.startswith(("RPS", "RPL"))
# hemoglobin genes
adata.var["hb"] = adata.var_names.str.contains("^HB[^(P)]")

In [ ]:
sc.pp.calculate_qc_metrics(
    adata, qc_vars=["mt", "ribo", "hb"], inplace=True, log1p=True
)

One can now inspect violin plots of some of the computed QC metrics:

* the number of genes expressed in the count matrix
* the total counts per cell
* the percentage of counts in mitochondrial genes

In [ ]:
adata

In [ ]:
sc.pl.violin(
    adata,
    ["n_genes_by_counts", "total_counts", "pct_counts_mt"],
    jitter=0.4,
    multi_panel=True,
)

Additionally, it is useful to consider QC metrics jointly by inspecting a scatter plot colored by `pct_counts_mt`. 

In [ ]:
sc.pl.scatter(adata, "total_counts", "n_genes_by_counts", color="pct_counts_mt")

Based on the QC metric plots, one could now remove cells that have too many mitochondrial genes expressed or too many total counts by setting manual or automatic thresholds. However, sometimes what appears to be poor QC metrics can be driven by real biology so we suggest starting with a very permissive filtering strategy and revisiting it at a later point. We therefore now only filter cells with less than 100 genes expressed and genes that are detected in less than 3 cells. 

Additionally, it is important to note that for datasets with multiple batches, quality control should be performed for each sample individually as quality control thresholds can very substantially between batches. 

In [ ]:
sc.pp.filter_cells(adata, min_genes=100)
sc.pp.filter_genes(adata, min_cells=3)

### Doublet detection

As a next step, we run a doublet detection algorithm. Identifying doublets is crucial as they can lead to misclassifications or distortions in downstream analysis steps. Scanpy contains the doublet detection method Scrublet {cite}`Wolock2019`. Scrublet predicts cell doublets using a nearest-neighbor classifier of observed transcriptomes and simulated doublets. {func}`scanpy.pp.scrublet` adds `doublet_score` and `predicted_doublet` to `.obs`. One can now either filter directly on `predicted_doublet` or use the `doublet_score` later during clustering to filter clusters with high doublet scores. 

In [ ]:
sc.pp.scrublet(adata, batch_key="sample")

We can remove doublets by either filtering out the cells called as doublets, or waiting until we've done a clustering pass and filtering out any clusters with high doublet scores.

:::{seealso}
Alternative methods for doublet detection within the scverse ecosystem are [DoubletDetection](https://github.com/JonathanShor/DoubletDetection) and [SOLO](https://docs.scvi-tools.org/en/stable/user_guide/models/solo.html). You can read more about these in the [Doublet Detection chapter](https://www.sc-best-practices.org/preprocessing_visualization/quality_control.html#doublet-detection) of Single Cell Best Practices.
:::

## Normalization

The next preprocessing step is normalization. A common approach is count depth scaling with subsequent log plus one (log1p) transformation. Count depth scaling normalizes the data to a “size factor” such as the median count depth in the dataset, ten thousand (CP10k) or one million (CPM, counts per million). The size factor for count depth scaling can be controlled via `target_sum` in `pp.normalize_total`. We are applying median count depth normalization with log1p transformation (AKA log1PF).

In [ ]:
# Saving count data
adata.layers["counts"] = adata.X.copy()

In [ ]:
# Normalizing to median total counts
sc.pp.normalize_total(adata)
# Logarithmize the data
sc.pp.log1p(adata)

## Feature selection

As a next step, we want to reduce the dimensionality of the dataset and only include the most informative genes. This step is commonly known as feature selection. The scanpy function `pp.highly_variable_genes` annotates highly variable genes by reproducing the implementations of Seurat {cite}`Satija2015`, Cell Ranger {cite}`Zheng2017`, and Seurat v3 {cite}`Stuart2019` depending on the chosen `flavor`. 

In [ ]:
sc.pp.highly_variable_genes(adata, n_top_genes=2000, batch_key="sample")

In [ ]:
sc.pl.highly_variable_genes(adata)

## Dimensionality Reduction
Reduce the dimensionality of the data by running principal component analysis (PCA), which reveals the main axes of variation and denoises the data.

In [ ]:
sc.tl.pca(adata)

Let us inspect the contribution of single PCs to the total variance in the data. This gives us information about how many PCs we should consider in order to compute the neighborhood relations of cells, e.g. used in the clustering function {func}`~scanpy.tl.leiden` or {func}`~scanpy.tl.tsne`. In our experience, there does not seem to be signifigant downside to overestimating the numer of principal components.

In [ ]:
sc.pl.pca_variance_ratio(adata, n_pcs=50, log=True)

You can also plot the principal components to see if there are any potentially undesired features (e.g. batch, QC metrics) driving signifigant variation in this dataset. In this case, there isn't anything too alarming, but it's a good idea to explore this.

In [ ]:
sc.pl.pca(
    adata,
    color=["sample", "sample", "pct_counts_mt", "pct_counts_mt"],
    dimensions=[(0, 1), (2, 3), (0, 1), (2, 3)],
    ncols=2,
    size=2,
)

## Nearest neighbor graph constuction and visualization

Let us compute the neighborhood graph of cells using the PCA representation of the data matrix.

In [ ]:
sc.pp.neighbors(adata)

This graph can then be embedded in two dimensions for visualiztion with UMAP (McInnes et al., 2018):

In [ ]:
sc.tl.umap(adata)

We can now visualize the UMAP according to the `sample`. 

In [ ]:
sc.pl.umap(
    adata,
    color="sample",
    # Setting a smaller point size to get prevent overlap
    size=2,
)

Even though the data considered in this tutorial includes two different samples, we only observe a minor batch effect and we can continue with clustering and annotation of our data. 

If you inspect batch effects in your UMAP it can be beneficial to integrate across samples and perform batch correction/integration. We recommend checking out [`scanorama`](https://github.com/brianhie/scanorama) and [`scvi-tools`](https://scvi-tools.org) for batch integration.

## Clustering

As with Seurat and many other frameworks, we recommend the Leiden graph-clustering method (community detection based on optimizing modularity) {cite}`Traag2019`. Note that Leiden clustering directly clusters the neighborhood graph of cells, which we already computed in the previous section.

In [ ]:
# Using the igraph implementation and a fixed number of iterations can be significantly faster, especially for larger datasets
sc.tl.leiden(adata, flavor="igraph", n_iterations=2)

In [ ]:
sc.pl.umap(adata, color=["leiden"])

## Re-assess quality control and cell filtering 

As indicated before, we will now re-assess our filtering strategy by visualizing different QC metrics using UMAP. 

In [ ]:
sc.pl.umap(
    adata,
    color=["leiden", "predicted_doublet", "doublet_score"],
    # increase horizontal space between panels
    wspace=0.5,
    size=3,
)

In [ ]:
sc.pl.umap(
    adata,
    color=["leiden", "log1p_total_counts", "pct_counts_mt", "log1p_n_genes_by_counts"],
    wspace=0.5,
    ncols=2,
)

## Manual cell-type annotation

:::{note}
This section of the tutorial is expanded upon using prior knowledge resources like automated assignment and gene enrichment in the scverse tutorial [here](https://scverse-tutorials.readthedocs.io/en/latest/notebooks/basic-scrna-tutorial.html#cell-type-annotation)
:::

Cell type annotation is laborous and repetitive task, one which typically requires multiple rounds of subclustering and re-annotation. It's difficult to show the entirety of the process in this tutorial, but we aim to show how the tools scanpy provides assist in this process.

We have now reached a point where we have obtained a set of cells with decent quality, and we can proceed to their annotation to known cell types. Typically, this is done using genes that are exclusively expressed by a given cell type, or in other words these genes are the marker genes of the cell types, and are thus used to distinguish the heterogeneous groups of cells in our data. Previous efforts have collected and curated various marker genes into available resources, such as [CellMarker](http://bio-bigdata.hrbmu.edu.cn/CellMarker/), [TF-Marker](http://bio.liclab.net/TF-Marker/), and [PanglaoDB](https://panglaodb.se/). The [cellxgene gene expression tool](https://cellxgene.cziscience.com/gene-expression) can also be quite useful to see which cell types a gene has been expressed in across many existing datasets.

Commonly and classically, cell type annotation uses those marker genes subsequent to the grouping of the cells into clusters. So, let's generate a set of clustering solutions which we can then use to annotate our cell types. Here, we will use the Leiden clustering algorithm which will extract cell communities from our nearest neighbours graph.

In [ ]:
for res in [0.02, 0.5, 2.0]:
    sc.tl.leiden(
        adata, key_added=f"leiden_res_{res:4.2f}", resolution=res, flavor="igraph"
    )

Notably, the number of clusters that we define is largely arbitrary, and so is the `resolution` parameter that we use to control for it. As such, the number of clusters is ultimately bound to the stable and biologically-meaningful groups that we can ultimately distringuish, typically done by experts in the corresponding field or by using expert-curated prior knowledge in the form of markers.

In [ ]:
sc.pl.umap(
    adata,
    color=["leiden_res_0.02", "leiden_res_0.50", "leiden_res_2.00"],
    legend_loc="on data",
)

In [ ]:
sc.pl.umap(
    adata,
    color=["leiden_res_0.02", "leiden_res_0.50", "leiden_res_2.00"],
    legend_loc="on data",
)

Though UMAPs should not be over-interpreted, here we can already see that in the highest resolution our data is over-clustered, while the lowest resolution is likely grouping cells which belong to distinct cell identities.

### Marker gene set

Let's define a set of marker genes for the main cell types that we expect to see in this dataset. These were adapted from [Single Cell Best Practices annotation chapter](https://www.sc-best-practices.org/cellular_structure/annotation.html), for a more detailed overview and best practices in cell type annotation, we refer the user to it.

In [ ]:
marker_genes = {
    "CD14+ Mono": ["FCN1", "CD14"],
    "CD16+ Mono": ["TCF7L2", "FCGR3A", "LYN"],
    # Note: DMXL2 should be negative
    "cDC2": ["CST3", "COTL1", "LYZ", "DMXL2", "CLEC10A", "FCER1A"],
    "Erythroblast": ["MKI67", "HBA1", "HBB"],
    # Note HBM and GYPA are negative markers
    "Proerythroblast": ["CDK6", "SYNGR1", "HBM", "GYPA"],
    "NK": ["GNLY", "NKG7", "CD247", "FCER1G", "TYROBP", "KLRG1", "FCGR3A"],
    "ILC": ["ID2", "PLCG2", "GNLY", "SYNE1"],
    "Naive CD20+ B": ["MS4A1", "IL4R", "IGHD", "FCRL1", "IGHM"],
    # Note IGHD and IGHM are negative markers
    "B cells": [
        "MS4A1",
        "ITGB1",
        "COL4A4",
        "PRDM1",
        "IRF4",
        "PAX5",
        "BCL11A",
        "BLK",
        "IGHD",
        "IGHM",
    ],
    "Plasma cells": ["MZB1", "HSP90B1", "FNDC3B", "PRDM1", "IGKC", "JCHAIN"],
    # Note PAX5 is a negative marker
    "Plasmablast": ["XBP1", "PRDM1", "PAX5"],
    "CD4+ T": ["CD4", "IL7R", "TRBC2"],
    "CD8+ T": ["CD8A", "CD8B", "GZMK", "GZMA", "CCL5", "GZMB", "GZMH", "GZMA"],
    "T naive": ["LEF1", "CCR7", "TCF7"],
    "pDC": ["GZMB", "IL3RA", "COBLL1", "TCF4"],
}

In [ ]:
sc.pl.dotplot(adata, marker_genes, groupby="leiden_res_0.02", standard_scale="var")

There are fairly clear patterns of expression for our markers show here, which we can use to label our coarsest clustering with broad lineages.

In [ ]:
adata.obs["cell_type_lvl1"] = adata.obs["leiden_res_0.02"].map(
    {
        "0": "Lymphocytes",
        "1": "Monocytes",
        "2": "Erythroid",
        "3": "B Cells",
    }
)

In [ ]:
sc.pl.dotplot(adata, marker_genes, groupby="leiden_res_0.50", standard_scale="var")

This seems like a resolution that suitable to distinguish most of the different cell types in our data. As such, let's try to annotate those by manually using the dotplot above, together with the UMAP of our clusters. Ideally, one would also look specifically into each cluster, and attempt to subcluster those if required.

### Differentially-expressed Genes as Markers

Furthermore, one can also calculate marker genes per cluster and then look up whether we can link those marker genes to any known biology, such as cell types and/or states. This is typically done using simple statistical tests, such as Wilcoxon and t-test, for each cluster vs the rest.

In [ ]:
# Obtain cluster-specific differentially expressed genes
sc.tl.rank_genes_groups(adata, groupby="leiden_res_0.50", method="wilcoxon")

We can then visualize the top 5 differentially-expressed genes on a dotplot.

In [ ]:
sc.pl.rank_genes_groups_dotplot(
    adata, groupby="leiden_res_0.50", standard_scale="var", n_genes=5
)

We can then use these genes to figure out what cell types we're looking at. For example, Cluster 7 is expressing [*NKG7*](https://www.genecards.org/cgi-bin/carddisp.pl?gene=NKG7&keywords=nkg7) and [*GNLY*](https://www.genecards.org/cgi-bin/carddisp.pl?gene=GNLY&keywords=GNLY), suggesting these are [NK cells](https://en.wikipedia.org/wiki/Natural_killer_cell).

To create your own plots, or use a more automated approach, the differentially expressed genes can be extracted in a convenient format with {func}`scanpy.get.rank_genes_groups_df`

In [ ]:
sc.get.rank_genes_groups_df(adata, group="7").head(5)

In [ ]:
dc_cluster_genes = sc.get.rank_genes_groups_df(adata, group="7").head(5)["names"]
sc.pl.umap(
    adata,
    color=[*dc_cluster_genes, "leiden_res_0.50"],
    legend_loc="on data",
    frameon=False,
    ncols=3,
)

You may have noticed that the p-values found here are extremely low. This is due to the statistical test being performed considering each cell as an independent sample. For a more conservative approach you may want to consider "pseudo-bulking" your data by sample (*e.g.* `sc.get.aggregate(adata, by=["sample", "cell_type"], func="sum", layer="counts")`) and using a more powerful differential expression tool, like [`pydeseq2`](https://pydeseq2.readthedocs.io/).

# HoloViz

In [ ]:
# adata.write(filename='adata.h5ad')
adata = ad.read_h5ad('adata.h5ad')

In [ ]:
adata.var_names

## Notes on anndata object:

adata 
- X : main data as n_obs × n_vars array = 17041 × 23427
- obs (observation/cell vectors as pandas Series): 'sample', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'n_genes', 'doublet_score', 'predicted_doublet', 'leiden', 'leiden_res_0.02', 'leiden_res_0.50', 'leiden_res_2.00', 'cell_type_lvl1'
- var (variable/gene vectors as pandas Series): 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection'
- uns (unstructured meta, config, stats as dicts): 'scrublet', 'log1p', 'hvg', 'pca', 'sample_colors', 'neighbors', 'umap', 'leiden', 'leiden_colors', 'predicted_doublet_colors', 'leiden_res_0.02', 'leiden_res_0.50', 'leiden_res_2.00', 'leiden_res_0.02_colors', 'leiden_res_0.50_colors', 'leiden_res_2.00_colors', 'rank_genes_groups', 'dendrogram_leiden_res_0.50'
- obsm (observation/cell matrices as n_obs x whatever array): 'X_pca', 'X_umap'
- varm (variable/gene matrices as n_var x whatever array): 'PCs'
- layers (array transforms, same size as main data): 'counts'
- obsp (paired observation/cell annotations as n_obs x n_obs array): 'distances', 'connectivities'
- var_names (variable/gene vectors as pandas Index): names of genes

In [ ]:
adata.uns['leiden_res_0.50']

## Convert to pandas

In [ ]:
import xarray as xr
import hvplot.xarray
import hvplot.pandas
import holoviews as hv
import pandas as pd
import numpy as np
from holoviews.operation.datashader import datashade, rasterize
hv.extension('bokeh')

In [ ]:
umap_df = pd.DataFrame(adata.obsm['X_umap'], columns=['UMAP1', 'UMAP2'])
pca_df = pd.DataFrame(adata.obsm['X_pca'], columns=[f'PCA{1+i}' for i in range(adata.obsm['X_pca'].shape[-1])])

obs_df = adata.obs.join(umap_df.set_index(adata.obs.index))
obs_df =  obs_df.join(pca_df.set_index(adata.obs.index))
var_df = adata.var.copy()

# Extract expression data for marker genes
sel_genes = marker_genes['CD16+ Mono'] #["TCF7L2", "FCGR3A", "LYN"],
expression_df = pd.DataFrame(
    adata[:, sel_genes].X.toarray(), 
    columns=sel_genes, 
    index=adata.obs_names
)

## Violins

In [ ]:
# later try to datshade jittered scatter as overlay using .opts(jitter=)
cols2plot = ["n_genes_by_counts", "total_counts", "pct_counts_mt"]
violins = []
for col in cols2plot:
    violins.append(obs_df[col].hvplot.violin(width=300, ylabel='Value', title=col))
hv.Layout(violins).opts(shared_axes=False)

## Scatters

In [ ]:
obs_df.hvplot.scatter(x="total_counts", y="n_genes_by_counts", color="pct_counts_mt", cmap='Viridis', rasterize=True, title='pct_counts_mt', aspect=1)

In [ ]:
# bug.. cannot get legend while using rasterize or datashade
# bug.. cannot set color_key while using 

## PCs

In [ ]:
pca1_2_count = obs_df.hvplot.scatter(x="PCA1", y="PCA2", color="pct_counts_mt", cmap='Viridis', title='pct_counts_mt', rasterize=True, cnorm='eq_hist', aspect=1)
pca1_2_samp = obs_df.hvplot.scatter(x="PCA1", y="PCA2", by="sample", legend=True, color=['blue', 'orange'], title='sample', datashade=True, cnorm='eq_hist', aspect=1)
# color_key={'s1d1':'blue', 's1d3':'orange'},
pca1_2_samp + pca1_2_count

## UMAP

In [ ]:
umap1_2_count = obs_df.hvplot.scatter(x="UMAP1", y="UMAP2", color="pct_counts_mt", cmap='Viridis', title='pct_counts_mt', rasterize=True, cnorm='eq_hist', aspect=1)
umap1_2_samp = obs_df.hvplot.scatter(x="UMAP1", y="UMAP2", by="sample", legend=True, color=['blue', 'orange'], title='sample', datashade=True, cnorm='eq_hist', aspect=1)
umap1_2_samp + umap1_2_count

In [ ]:
import datashader as ds

In [ ]:
points = hv.Points(obs_df, kdims=['UMAP1', 'UMAP2'], vdims=['leiden_res_0.50'])
umap1_2_lieden_hv = rasterize(points, aggregator=ds.by('leiden_res_0.50')).opts(
        cmap='Category20', frame_height=300, aspect=1, tools=['hover'], title='leiden, HoloViews')

umap1_2_lieden_hvplot = obs_df.hvplot.scatter(x="UMAP1", y="UMAP2", by="leiden_res_0.50", cmap='Category20', title='leiden, hvPlot', datashade=True, cnorm='eq_hist', aspect=1)
umap1_2_lieden_hv + umap1_2_lieden_hvplot

## dotplot with dendrogram

In [ ]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from scipy.cluster.hierarchy import linkage, dendrogram
from scipy.spatial.distance import pdist
import holoviews as hv
from holoviews import opts
import panel as pn
pn.extension()
hv.extension('bokeh')

def compute_dotplot_data(expression, groupby, gene_names, markers, expression_cutoff):
    """
    Compute data required for creating a dot plot, handling markers as a list or a dictionary,
    and allowing for genes to appear in multiple groups.

    Parameters:
    - expression: n_cells x n_genes matrix (could be sparse)
    - groupby: array of cluster assignments, length n_cells
    - gene_names: list of gene names, length n_genes
    - markers: either a list of gene names or a dictionary with keys as group labels and values as lists of gene names
    - expression_cutoff: value used for binarizing expression (expression > cutoff)

    Returns:
    - df: DataFrame with columns ['gene', 'cluster', 'percentage', 'mean_expression', 'gene_group']
    """
    if not isinstance(expression, csr_matrix):
        expression = csr_matrix(expression)

    gene_name_to_idx = {gene: idx for idx, gene in enumerate(gene_names)}

    gene_group_list = []

    if isinstance(markers, dict):
        # Flatten markers dictionary to get list of (gene, group)
        marker_genes = []
        for group, genes in markers.items():
            for gene in genes:
                if gene in gene_name_to_idx:
                    marker_genes.append((gene, group))

        # Get the list of gene indices, allowing for duplicates
        marker_indices = [gene_name_to_idx[gene] for gene, group in marker_genes]
        marker_gene_names = [gene for gene, group in marker_genes]
        gene_groups = [group for gene, group in marker_genes]

    elif isinstance(markers, list):
        marker_genes = [gene for gene in markers if gene in gene_name_to_idx]

        # Remove duplicates while preserving order
        marker_genes = list(dict.fromkeys(marker_genes))

        marker_indices = [gene_name_to_idx[gene] for gene in marker_genes]
        marker_gene_names = marker_genes
        gene_groups = [None] * len(marker_gene_names)

    else:
        raise ValueError("Markers must be a list or a dictionary.")

    groupby = np.array(groupby)
    clusters = np.unique(groupby)

    clusters_series = pd.Series(clusters)
    clusters_numeric = pd.to_numeric(clusters_series, errors='coerce')

    if clusters_numeric.isnull().any():
        convert_cluster_to_numeric = False
    else:
        convert_cluster_to_numeric = True

    results = []

    for gene_idx, gene_name, gene_group in zip(marker_indices, marker_gene_names, gene_groups):
        gene_expression = expression[:, gene_idx]

        gene_expression_binarized = gene_expression.copy()
        gene_expression_binarized.data = (gene_expression_binarized.data > expression_cutoff).astype(int)

        for cluster in clusters:
            cluster_mask = groupby == cluster
            cluster_cell_indices = np.where(cluster_mask)[0]

            n_cells_in_cluster = len(cluster_cell_indices)

            X_cluster = gene_expression[cluster_cell_indices]
            X_cluster_binarized = gene_expression_binarized[cluster_cell_indices]

            # sum of binarized expression (number of cells expressing the gene)
            expressing_cells = X_cluster_binarized.sum()

            percentage = (expressing_cells / n_cells_in_cluster) * 100

            # mean expression per gene over all cells in cluster
            total_expression = X_cluster.sum()
            mean_expression = total_expression / n_cells_in_cluster

            cluster_results = pd.DataFrame({
                'gene': [gene_name],
                'cluster': [cluster],
                'percentage': [percentage],
                'mean_expression': [mean_expression],
                'gene_group': [gene_group]
            })
            results.append(cluster_results)

    df = pd.concat(results, ignore_index=True)

    if convert_cluster_to_numeric:
        df['cluster'] = pd.to_numeric(df['cluster'])

    return df

def plot_dotplot_w_dendro(df, max_point_size=40):
    """
    Create a dot plot from the DataFrame output by compute_dotplot_data,
    handling genes that appear multiple times due to being in multiple groups,
    and adding a dendrogram to the right of the main plot.

    Parameters:
    - df: DataFrame with columns ['gene', 'cluster', 'percentage', 'mean_expression', 'gene_group']
    - max_point_size: Maximum size for the dots representing 100% percentage.

    Returns:
    - plot: Holoviews object representing the dot plot with a dendrogram.
    """

    # unique identifier for each gene entry
    df = df.copy()
    df['gene_id'] = df.apply(
        lambda row: f"{row['gene']} ({row['gene_group']})" if pd.notnull(row['gene_group']) else row['gene'],
        axis=1
    )

    # percentage to point sizes
    df['size'] = (df['percentage'] / df['percentage'].max()) * max_point_size

    # for color mapping
    df['mean_expression_normalized'] = df['mean_expression'] / df['mean_expression'].max()

    gene_ids_order = df['gene_id'].drop_duplicates().tolist()
    df['gene_id'] = pd.Categorical(df['gene_id'], categories=gene_ids_order, ordered=True)
    cluster_gene_matrix = df.pivot_table(index='cluster', columns='gene_id', values='mean_expression', fill_value=0)

    # hierarchical clustering
    X = cluster_gene_matrix.values
    cluster_dist = pdist(X, metric='euclidean')
    cluster_linkage = linkage(cluster_dist, method='average')

    dendro_data = dendrogram(cluster_linkage, labels=cluster_gene_matrix.index, no_plot=True)
    clusters_ordered = dendro_data['ivl']

    # Update df['cluster'] to be categorical with the new order
    df['cluster'] = pd.Categorical(df['cluster'], categories=clusters_ordered, ordered=True)

    # Map cluster labels to positions (for y-axis)
    cluster_positions = {cluster: pos for pos, cluster in enumerate(clusters_ordered)}
    df['cluster_pos'] = df['cluster'].map(cluster_positions)

    cluster_values = sorted(df['cluster'].unique())

    min_cluster = min(cluster_values)
    max_cluster = max(cluster_values)
    ylim = (max_cluster + 0.5, min_cluster - 0.5)

    yticks = [(cluster, str(cluster)) for cluster in cluster_values]

    points = hv.Points(
        df,
        kdims=['gene_id', 'cluster_pos'],
        vdims=['mean_expression_normalized', 'size', 'percentage', 'mean_expression', 'gene_group']
    )

    points = points.opts(
        xrotation=90,
        color='mean_expression_normalized',
        cmap='Reds',
        size='size',
        line_color='black',
        line_alpha=0.1,
        marker='o',
        tools=['hover'],
        colorbar=True,
        width=1000,
        frame_height=400,
        # responsive=True,
        xlabel='Gene',
        ylabel='Cluster',
        invert_yaxis=False,
        show_legend=False,
        yticks=yticks,
        ylim=ylim,
    )

    if 'gene_group' in df.columns:
        gene_groups = df[['gene_id', 'gene_group']].drop_duplicates()
        gene_groups['group_code'] = gene_groups['gene_group'].factorize()[0]

        annotations_df = gene_groups[['gene_id', 'group_code', 'gene_group']].copy()
        annotations_df['Group'] = 'Group'

        annotations_df['gene_id'] = pd.Categorical(annotations_df['gene_id'], categories=gene_ids_order, ordered=True)
        annotations_df['Group'] = pd.Categorical(annotations_df['Group'], categories=['Group'], ordered=True)

        annotations_heatmap = hv.HeatMap(
            annotations_df,
            kdims=['gene_id', 'Group'],
            vdims=['group_code', 'gene_group']
        )

        annotations_heatmap = annotations_heatmap.opts(
            colorbar=False,
            xaxis=None,
            yaxis=None,
            responsive=False,
            width=600,
            cmap='glasbey_hv',
            tools=['hover'],
            toolbar=None,
            height=50,
            show_frame=False,
            show_grid=False,
        )

        main_plot = hv.Layout([annotations_heatmap, hv.Empty(), points]).cols(2).opts(
            opts.Layout(shared_axes=True)
        )
    else:
        main_plot = points

    # Mapping leaf positions to cluster positions
    leaf_positions = {int(leaf_id): cluster_positions[clusters_ordered[int(leaf_id)]] for leaf_id in dendro_data['leaves']}

    # What's going on? trying a swap
    dendro_paths = []
    icoord = np.array(dendro_data['dcoord'])
    dcoord = np.array(dendro_data['icoord'])

    for xs, ys in zip(icoord, dcoord):
        ys_new = []
        for y in ys:
            if y % 10 == 5.0:
                # Leaf node
                leaf_id = int((y - 5.0) / 10.0)
                ys_new.append(leaf_positions[leaf_id])
            else:
                # Internal node
                ys_new.append(y / max(dcoord.flatten()) * (len(clusters_ordered) - 1))
        dendro_paths.append(np.column_stack([xs, ys_new]))

    dendrogram_plot = hv.Path(dendro_paths, ['Distance', 'Cluster'])

    dendrogram_plot = dendrogram_plot.opts(
        width=200,
        frame_height=400,
        xlabel='',
        # ylabel='',
        invert_yaxis=False,
        xaxis=None,
        yaxis='right',
        # yaxis=None,
        show_frame=False,
        # show_grid=False,
        fontsize={'labels': '8pt'},
        tools=['hover'],
        yticks=yticks,
        ylim=ylim,
        
    )

    return (points + dendrogram_plot).opts(shared_axes=True)


In [ ]:
dp_data = compute_dotplot_data(adata.X, adata.obs['leiden_res_0.50'], adata.var_names, marker_genes, 0.1)
plot_dotplot_w_dendro(dp_data, max_point_size=12)

## dotplot with annotation bar

In [ ]:
def plot_dotplot_w_bar(df, max_point_size=40):
    """
    Create a dot plot from the DataFrame output by compute_dotplot_data,
    handling genes that appear multiple times due to being in multiple groups.

    Parameters:
    - df: DataFrame with columns ['gene', 'cluster', 'percentage', 'mean_expression', 'gene_group']
    - max_point_size: Maximum size for the dots representing 100% percentage.

    Returns:
    - plot: Holoviews object representing the dot plot.
    """
    df = df.copy()
    df['gene_id'] = df.apply(
        lambda row: f"{row['gene']} ({row['gene_group']})" if pd.notnull(row['gene_group']) else row['gene'],
        axis=1
    )

    df['size'] = (df['percentage'] / df['percentage'].max()) * max_point_size
    df['mean_expression_normalized'] = df['mean_expression'] / df['mean_expression'].max()
    gene_ids_order = df['gene_id'].drop_duplicates().tolist()
    df['gene_id'] = pd.Categorical(df['gene_id'], categories=gene_ids_order, ordered=True)
    cluster_values = sorted(df['cluster'].unique())
    min_cluster = min(cluster_values)
    max_cluster = max(cluster_values)
    ylim = (max_cluster + 0.5, min_cluster - 0.5)
    yticks = [(cluster, str(cluster)) for cluster in cluster_values]
    points = hv.Points(
        df,
        kdims=['gene_id', 'cluster'],
        vdims=['mean_expression_normalized', 'size', 'percentage', 'mean_expression', 'gene_group']
    )

    points = points.opts(
        xrotation=90,
        color='mean_expression_normalized',
        cmap='Reds',
        size='size',
        line_color='black',
        line_alpha=0.1,
        marker='o',
        tools=['hover'],
        colorbar=True,
        min_height=400,
        responsive=True,
        xlabel='Gene',
        ylabel='Cluster',
        invert_yaxis=False,
        show_legend=False,
        yticks=yticks,
        ylim=ylim,
    )

    if 'gene_group' in df.columns:
        gene_groups = df[['gene_id', 'gene_group']].drop_duplicates()
        gene_groups['group_code'] = gene_groups['gene_group'].factorize()[0]

        annotations_df = gene_groups[['gene_id', 'group_code', 'gene_group']].copy()
        annotations_df['Group'] = 'Group'

        annotations_df['gene_id'] = pd.Categorical(annotations_df['gene_id'], categories=gene_ids_order, ordered=True)
        annotations_df['Group'] = pd.Categorical(annotations_df['Group'], categories=['Group'], ordered=True)

        annotations_heatmap = hv.HeatMap(
            annotations_df,
            kdims=['gene_id', 'Group'],
            vdims=['group_code', 'gene_group']
        )

        annotations_heatmap = annotations_heatmap.opts(
            colorbar=False,
            xaxis=None,
            yaxis=None,
            responsive=True,
            cmap='glasbey_hv',
            tools=['hover'],
            toolbar=None,
            height=50,
            show_frame=False,
            show_grid=False,
        )

        layout = hv.Layout([annotations_heatmap, points]).cols(1).opts(
            opts.Layout(shared_axes=True)
        )
        return layout

    else:
        return points


In [ ]:
dp_data = compute_dotplot_data(adata.X, adata.obs['leiden_res_0.50'], adata.var_names, marker_genes, .1)
plot_dotplot_w_bar(dp_data, max_point_size=12)
